In [12]:
from datascience import *
import numpy as np
import pandas as pd
import json

In [13]:
def truncate_names(name):
    truncated = name.split(" ")
    modified = 0
    
    # abbrev
    if np.all([True if x.isupper() else False for x in truncated[0]]):
        if truncated[1][0] == truncated[0][0]:
            truncated = truncated[1:]
            modified = 1
        else:
            modified = 0
    has_p = [True if x.find("(") != -1 else False for x in truncated]
    if np.any(has_p):
        index = ([i for i, x in enumerate(has_p) if x][0])
        truncated = " ".join(truncated[0:index])
        modified = 1
    
    if not modified:
        return name
    return " ".join(truncated) if type(truncated) == list else truncated

def nodes_links(table, mapping):
    # NODES
    industry_json = {}
    nodes = []
    links = []
    nodes_details = {}
    sponsors = table.column('Sponsor')
    depts = table.column('Department')
    total = table.column('Total')
    index = 0
    for i in range(table.num_rows):
        node_json = {}
        if sponsors[i] not in nodes_details.keys():
            nodes_details[sponsors[i]] = index
            node_json['name'] = sponsors[i]
            vals = [(val['abbrev'], val['cat']) for val in mappings['nodes'] if node_json['name'] == val['name']]
            node_json['type'] = "sponsors"
            if len(vals):
                node_json['cat'] = vals[0][1]
                node_json['abbrev'] = vals[0][0]
            else:
                node_json['cat'] = ""
                node_json['abbrev'] = ""
            index += 1
            nodes = nodes + [node_json]
        node_json1 = {}
        if depts[i] not in nodes_details.keys():
            nodes_details[depts[i]] = index
            node_json1['name'] = depts[i]
            vals = [(val['abbrev'], val['cat']) for val in mappings['nodes'] if node_json1['name'] == val['name']]
            node_json1['type'] = "targets"
            if len(vals):
                node_json1['cat'] = vals[0][1]
                node_json1['abbrev'] = vals[0][0]
            else:
                node_json1['cat'] = ""
                node_json1['abbrev'] = ""
            index += 1
            nodes = nodes + [node_json1]
    # LINKS
    for i in range(table.num_rows):
        link_json = {}
        link_json['source'] = nodes_details[sponsors[i]]
        link_json['target'] = nodes_details[depts[i]]
        value = total[i]
        if value > 0: 
            link_json['with_neg'] = total[i]
            link_json['value'] = total[i]
        else:
            link_json['with_neg'] = value
            link_json['value'] = 0
        links += [link_json]
    industry_data = {}
    industry_data['nodes'] = nodes
    industry_data['links'] = links
    return industry_data

In [145]:
# data/research_funding_clean.csv is w/o additional wellspring
# data/research_funding_clean_wwells.csv is w/ additional wellspring
combined_phoebe_conc_pd_ds = Table().read_table('data/research_funding_clean.csv')
combined_phoebe_conc_pd_ds.show(5)

Unnamed: 0,Sponsor,﻿Control Unit,Division,Department,class_clean,2014,2015,2016
0,"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,0,0,0
1,3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,0,250000,0
2,"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,22408
3,"AFINGEN, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,5000
4,"APATH, L.C.C.",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Industry,0,0,0


In [146]:
combined_phoebe_conc_pd_ds.relabel('class_clean', 'Sponsor Class')

Unnamed: 0,Sponsor,﻿Control Unit,Division,Department,Sponsor Class,2014,2015,2016
0,"""DEACTIVATED - use 00V8VH"" NIH National Center for Resea ...",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Federal,0,0,0
1,3M Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,Industry,0,250000,0
2,"4D Molecular Therapeutics, LLC",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,22408
3,"AFINGEN, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),Industry,0,0,5000
4,"APATH, L.C.C.",L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Industry,0,0,0
5,AYUSA International,Vice Chancellor - Research,Academic Research Units,IURD Institute of Urban & Regional Development,Not for Profit,54481,60000,0
6,Aalborg University Copenhagen,Vice Chancellor - Research,Vice Chancellor for Research ORUs,IBER Institute of Business & Economic Research,Not for Profit,0,53015,0
7,Abraham J. & Phyllis Katz Foundation,L&S College of Letters & Sciences,L&S Biological Sciences,MCB Molecular & Cell Biology,Not for Profit,0,0,0
8,"Abt Associates, Inc.",Colleges,CNR College of Natural Resources,Null,Industry,73942,0,0
9,Academy of Applied Science,Colleges,CNR College of Natural Resources,"Environmental Science, Policy & Management",Not for Profit,0,4000,4000


In [147]:
industry = combined_phoebe_conc_pd_ds.where('Sponsor Class', "Industry")
industry = industry.with_column("Total", industry.apply(lambda x, y, z: x+y+z, ['2014', '2015', '2016']))\
                    .select([1, 2, 3, 4, 'Total'])\
                    .sort('Total', descending=True)
#                     .where('Total', are.above(0))
industry.show(5)

Sponsor,﻿Control Unit,Division,Department,Total
BASF Corporation,Colleges,College of Chemistry,Chemistry Department,9069312
Aduro Biotech,L&S College of Letters & Sciences,L&S Biological Sciences,Cancer Research Laboratory,7500000
"BP Group (BP America, BP Exploration, British Petroleum)",Vice Chancellor - Research,VC Res Other Research Units,EBI Energy Biosciences Institute,5000000
Intel Corporation,Colleges,College of Engineering,EECS Electrical Engineering & Computer Sciences,3929000
"Pfizer, Inc.",Vice Chancellor - Research,VC Res Other Research Units,The California Institute for Quantitative Biosciences (QB3),2904754


In [148]:
industry_nonnull = industry.where(np.logical_and(industry.column("Department") != "nan", industry.column("Department") != "Null"))\
                           .select(['Sponsor','Department', 'Total'])
        
industry_null = industry.where('Department', 'Null') \
                        .append(industry.where("Department", 'nan')) \
                        .select(["Sponsor", 'Division', "Total"]) \
                        .relabel("Division", "Department")
            
industry_all = industry_nonnull.append(industry_null)
industry_all = industry_all.with_columns(['Sponsor', 
                                        industry_all.apply(truncate_names, ['Sponsor']),
                                         'Department',
                                        industry_all.apply(truncate_names, ['Department'])])
industry_all.show(3)

Sponsor,Department,Total
BASF Corporation,Chemistry Department,9069312
Aduro Biotech,Cancer Research Laboratory,7500000
BP Group,Energy Biosciences Institute,5000000


In [149]:
top50_sponsors = industry_all.group('Sponsor', sum)\
                            .sort('Total sum', descending=True)\
                            .take(np.arange(50))\
                            .column('Sponsor')

In [150]:
top50_sponsors

array(['BASF Corporation', 'Aduro Biotech', 'Intel Corporation',
       'BP Group', 'Pfizer, Inc.', 'Chevron Corporation',
       'Samsung Electronics Company, Ltd.', 'Toyota Motor Corporation',
       'Dow Chemical Company', 'AstraZeneca Inc.', 'Qualcomm, Inc.',
       'GlaxoSmithKline', 'Wellspring Advisors LLC',
       'Hoffmann-La Roche LTD.', 'Altria Group, Inc.', 'BAE Systems',
       'TATA Consultancy Services', 'Volkswagen Group of America, Inc.',
       'Robert Bosch Corporation GMBH', 'Ford Motor Company',
       'Panasonic Corporation', 'Charles River Analytics',
       'Taiwan Semiconductor Manufacturing Company. Ltd',
       'State Street Corporation', 'Siemens Corporation',
       'InvenSense, Inc.', 'Korea Electric Power Corporation',
       'Google, Inc.', 'Sandia National Laboratory',
       'International Business & Technical Consultants, Inc.',
       'Denso Corporation', 'Exxon Mobil Corporation',
       'NVIDIA Corporation', 'Cray Research, Inc.',
       'Samsung S

In [151]:
industry_all = industry_all.with_column('Sponsor Copy', industry_all.column('Sponsor'))
industry_all = industry_all.with_column('Sponsor',
                           industry_all.apply(lambda x: x if x in top50_sponsors else 'Other', ['Sponsor']))
industry_all

Sponsor,Department,Total,Sponsor Copy
BASF Corporation,Chemistry Department,9069312,BASF Corporation
Aduro Biotech,Cancer Research Laboratory,7500000,Aduro Biotech
BP Group,Energy Biosciences Institute,5000000,BP Group
Intel Corporation,Electrical Engineering & Computer Sciences,3929000,Intel Corporation
"Pfizer, Inc.",The California Institute for Quantitative Biosciences,2904754,"Pfizer, Inc."
Chevron Corporation,Chemical and Biomolecular Engineering,2825291,Chevron Corporation
Intel Corporation,Engineering Research Support Organization,2409614,Intel Corporation
AstraZeneca Inc.,Innovative Genomics Initiative,2340000,AstraZeneca Inc.
Wellspring Advisors LLC,Center for Effective Global Action,1879998,Wellspring Advisors LLC
GlaxoSmithKline,Chemical and Biomolecular Engineering,1810954,GlaxoSmithKline


In [152]:
industry_all.group('Sponsor', sum).sort('Total sum', descending=True).show(4)

Sponsor,Department sum,Total sum,Sponsor Copy sum
Other,,24977243,
BASF Corporation,,10739602,
Aduro Biotech,,7926154,
Intel Corporation,,6338614,


In [153]:
# Making one aggregate Other link for each Department
industry_grouped_other = industry_all.where('Sponsor', 'Other')\
                                     .group('Department', sum)\
                                     .relabel('Total sum', 'Total')\
                                     .select(['Department', 'Total'])
industry_grouped_other = industry_grouped_other.with_column('Sponsor', 
                                                    ['Other']*industry_grouped_other.num_rows)

# Combining top50 with the others 
industry_grouped_all = industry_all.where(industry_all.column('Sponsor') != "Other")\
                                   .drop('Sponsor Copy')\
                                   .append(industry_grouped_other)
    
industry_grouped_all = industry_grouped_all.groups(['Sponsor', 'Department'], sum).sort('Total sum', descending=True)\
                                            .relabel('Total sum', 'Total')
industry_grouped_all

Sponsor,Department,Total
BASF Corporation,Chemistry Department,9069312
Aduro Biotech,Cancer Research Laboratory,7500000
Other,Electrical Engineering & Computer Sciences,6741182
BP Group,Energy Biosciences Institute,5000000
Intel Corporation,Electrical Engineering & Computer Sciences,3929000
"Pfizer, Inc.",The California Institute for Quantitative Biosciences,2904754
Chevron Corporation,Chemical and Biomolecular Engineering,2825291
Intel Corporation,Engineering Research Support Organization,2409614
AstraZeneca Inc.,Innovative Genomics Initiative,2340000
Wellspring Advisors LLC,Center for Effective Global Action,1879998


In [154]:
# Mapping the smaller donors in 'Others'
industry_other_grouped = industry_all.where(industry_all.column("Total") != 0)\
                                    .where('Sponsor', 'Other')\
                                     .groups(['Department', 'Sponsor Copy'], sum)\
                                     .drop(['Sponsor sum'])\
                                    .sort('Total sum', descending=True)
    
industry_other_grouped

Department,Sponsor Copy,Total sum
Electrical Engineering & Computer Sciences,GlobalFoundries U.S. INC.,640000
Electrical Engineering & Computer Sciences,General Electric Company,630124
Engineering Research Support Organization,Texas Instruments,550000
SPH Divisional Rsrch and Cntrs,"The Mitchell Group, Inc.",541961
"Environmental Science, Policy & Management","Aerosol Dynamics, Inc.",537512
SPH Divisional Rsrch and Cntrs,"Newomics, Inc.",494601
Molecular & Cell Biology,Innate Pharma SAS,462000
Materials Science & Engineering,General Research Institute For Nonferrous Metals,450000
Space Sciences Lab,"NOVA Scientific, Inc.",412281
"Dean's Office, Public Health","Takeda Vaccines, Inc.",400074


In [160]:
# (cont.)
mappings = json.loads(open("data/node_dictionary.json").read())

def other_mapping_func(table, mappings, abbrev=True):
    
    other_mapping = {}

    targets = table.column('Department')
    sponsors = table.column('Sponsor Copy')
    total = table.column('Total sum')
    for i in range(table.num_rows):
        sub = {}
        sub['sponsor'] = sponsors[i]
        sub['value'] = total[i]
        target = targets[i]
        target_alt_name = [val['abbrev'] for val in mappings['nodes'] if target == val['name']]
        if abbrev and target_alt_name and len(target_alt_name[0]) > 5:
            target = target_alt_name[0]
        if target in other_mapping.keys():
            other_mapping[target] += [sub]
        else:
            other_mapping[target] = [sub]
    return other_mapping
other_mapping_func(industry_other_grouped, mappings, True)  # JSON

{'Berkeley Seismological Laboratory': [{'sponsor': 'Array Information Technology, Inc.',
   'value': 213855}],
 'Bioengineering': [{'sponsor': 'Novartis Institutes for BioMedical Research',
   'value': 377099},
  {'sponsor': 'Zymergen, Inc.', 'value': 125600},
  {'sponsor': 'Microbial Medical, Inc.', 'value': 22500},
  {'sponsor': 'OncoSec Medical Inc.', 'value': 15650}],
 'Botanical Garden': [{'sponsor': 'Vollmar Natural Lands Consulting',
   'value': 41000},
  {'sponsor': 'Creekside Center for Earth Observation', 'value': 11200}],
 'California Institute for Energy & Environment': [{'sponsor': 'Food Service Technology Center',
   'value': 65000},
  {'sponsor': 'Alstom', 'value': 50000}],
 'California Institute for Quantitative Biosciences': [{'sponsor': 'Anacor Pharmaceuticals, Inc.',
   'value': 342139},
  {'sponsor': "L'Oreal", 'value': 274000},
  {'sponsor': 'Leaf Technologies', 'value': 150000},
  {'sponsor': 'Dow Agrosciences LLC', 'value': 150000},
  {'sponsor': 'BioInspira, Inc

In [156]:
# For TOP6 Breakout Sankey

# top6 = top50_sponsors[0:6]
# top6_table = industry_json_table.with_column("top6", 
#                                            industry_json_table.apply(lambda x: x in top6, ['Sponsor']))\
#                                 .where('top6', True)
# nodes_links(top6_table, mappings)

In [157]:
# For (mobile) sankey
mob_sankey = industry_all.groups(['Sponsor Copy', 'Department'], sum)\
                        .drop('Sponsor sum')
mob_sankey = mob_sankey.where(mob_sankey.column('Total sum') != 0).sort('Total sum', descending=True)
# other_mapping_func(mob_sankey)
other_mapping_func(mob_sankey, mappings)
# 10/12 10:18 this is w/o wellspring

{'Berkeley Seismological Laboratory': [{'sponsor': 'Array Information Technology, Inc.',
   'value': 213855}],
 'Bioengineering': [{'sponsor': 'Novartis Institutes for BioMedical Research',
   'value': 377099},
  {'sponsor': 'Zymergen, Inc.', 'value': 125600},
  {'sponsor': 'Microbial Medical, Inc.', 'value': 22500},
  {'sponsor': 'OncoSec Medical Inc.', 'value': 15650}],
 'Botanical Garden': [{'sponsor': 'Vollmar Natural Lands Consulting',
   'value': 41000},
  {'sponsor': 'Creekside Center for Earth Observation', 'value': 11200}],
 'California Institute for Energy & Environment': [{'sponsor': 'Pacific Gas and Electric Company',
   'value': 450850},
  {'sponsor': 'Food Service Technology Center', 'value': 65000},
  {'sponsor': 'Alstom', 'value': 50000}],
 'California Institute for Quantitative Biosciences': [{'sponsor': 'Pfizer, Inc.',
   'value': 2904754},
  {'sponsor': 'Hoffmann-La Roche LTD.', 'value': 1778482},
  {'sponsor': 'TATA Consultancy Services', 'value': 1502000},
  {'spon

In [158]:
industry_json_table = industry_grouped_all.where(industry_grouped_all.column("Total") != 0).sort('Total', descending=True)
industry_json = nodes_links(industry_json_table, mappings) # JSON 

In [159]:
industry_json

{'links': [{'source': 0, 'target': 1, 'value': 9069312, 'with_neg': 9069312},
  {'source': 2, 'target': 3, 'value': 7500000, 'with_neg': 7500000},
  {'source': 4, 'target': 5, 'value': 6741182, 'with_neg': 6741182},
  {'source': 6, 'target': 7, 'value': 5000000, 'with_neg': 5000000},
  {'source': 8, 'target': 5, 'value': 3929000, 'with_neg': 3929000},
  {'source': 9, 'target': 10, 'value': 2904754, 'with_neg': 2904754},
  {'source': 11, 'target': 12, 'value': 2825291, 'with_neg': 2825291},
  {'source': 8, 'target': 13, 'value': 2409614, 'with_neg': 2409614},
  {'source': 14, 'target': 15, 'value': 2340000, 'with_neg': 2340000},
  {'source': 16, 'target': 17, 'value': 1879998, 'with_neg': 1879998},
  {'source': 18, 'target': 12, 'value': 1810954, 'with_neg': 1810954},
  {'source': 19, 'target': 10, 'value': 1778482, 'with_neg': 1778482},
  {'source': 20, 'target': 21, 'value': 1725779, 'with_neg': 1725779},
  {'source': 22, 'target': 1, 'value': 1724999, 'with_neg': 1724999},
  {'source

In [120]:
def get_val(source_id):
    sum = 0
    for elem in industry_json['links']:
        if elem['source'] == source_id:
            sum += elem['with_neg']
    return sum
    
agg_sponsortypes = {}
num = 0
for i in range(len(industry_json["nodes"])):
    elem = industry_json["nodes"][i]    
    if elem['name'] != "Other" and elem['type'] == "sponsors":
        num += 1
        val = get_val(i)
        if (not elem['cat']):
            print(elem)
        if elem['cat'] not in agg_sponsortypes.keys():
            agg_sponsortypes[elem['cat']] = val
        else:
            agg_sponsortypes[elem['cat']] += val

In [119]:
agg_sponsortypes

{'Aerospace': 1640860,
 'Automotives': 7616473,
 'Biotechnology': 7926154,
 'Chemicals': 13454726,
 'Consultancy': 6909181,
 'Energy': 13345989,
 'Optometry': 660096,
 'Pharmaceuticals': 11460542,
 'Technology': 36163218,
 'Tobacco': 1725779}

In [56]:
# if division or department includes sciences, chemistry, engineering, transportation, 
def non_stem(div, dept, nsf=True):
    if nsf:
        keywords = ['education', 'haas',
                   'regional development', 'humanities']
        not_ = []
    else:
        keywords = ['social sciences', 'international studies', 'education', 'labor and employment',
               'law', 'haas', 'regional development', 'humanities', 'economics', 'public policy'] 
        not_ = []
    
    div_ = np.any([x in div.lower() for x in keywords])
    dept_ = np.any([x in dept.lower() for x in keywords])
    return (div_ or dept_) and ((div.lower() not in not_) and (dept.lower() not in not_))

In [57]:
stem_industry_common = industry.with_column('Non_Stem',
                        industry.apply(lambda x, y: non_stem(x, y, False), ['Division', 'Department']))
stem_industry_NSF = industry.with_column('Non_Stem',
                        industry.apply(lambda x, y: non_stem(x, y), ['Division', 'Department']))

In [58]:
non = sum(stem_industry_common.where('Non_Stem', True).column('Total'))
stem = sum(stem_industry_common.where('Non_Stem', False).column('Total'))
print("for common understanding of STEM")
print(non, "to non-stem from industry")
print(stem, "to stem from industry")
print(non/stem ,": 1 stem to non-stem ratio")


for common understanding of STEM
3444528 to non-stem from industry
122435733 to stem from industry
0.0281333554805 : 1 stem to non-stem ratio
------------------------
for NSF def of STEM
306409 to non-stem from industry
125573852 to stem from industry
409.824293673 : 1 stem to non-stem ratio
